In [ ]:
import numpy as np
import pandas as pd
import xlrd
import openpyxl

import os
folder_name = '应收帐款汇总表'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

import sys
month=' ('+str(sys.argv[1])+'月)'
    
file_name = '应收帐款汇总表'
input_filename = file_name +'.xls'
output_filename = folder_name + '/'+file_name+'_本期增加.xlsx'

df = pd.read_excel(input_filename, sheet_name =0)
df.columns.values

In [ ]:
names = df.loc[0]
print(names)
print(names[names == '名称'].index)
print(names[names == '本期增加'].index)

In [ ]:
df.columns = df.iloc[0,:]
print(df.columns.values)

df2 = df[ df['本期增加'] != 0]
df2 = df2[ ['名称','本期增加'] ]
df2.head()

In [ ]:
df2.set_index('名称',inplace=True)
df2.reset_index(inplace=True)
#df2.set_value(df2.shape[0]-1, '名称', '总计')
df2.iat[df2.shape[0]-1,0] ='总计'
df2.drop([0],axis=0,inplace=True)
df2.index.names=['序号']


In [ ]:
df2.head()

In [ ]:
df2.tail()

In [ ]:
def handle_worksheet_format(writer, group, title):
    # handle the spreadsheet format
    workbook = writer.book
    worksheet = writer.sheets['sheet1']      
    # set data format
    for idx, col in enumerate(group):  # loop through all columns
        series = group[col]
        if isinstance(series.iloc[0],str): char_ratio = 2 
        else: char_ratio = 1 
        max_len = max((
                        series.astype(str).map(len).max() * char_ratio ,  # len of largest item
                        len( str(series.name) * 2)  # len of column name/header
                    )) + 2  # adding a little extra space

        data_fmt = workbook.add_format({'num_format': '0.00',
                                        'align': 'right'})    
        worksheet.set_column(idx+1, idx+1, max_len, data_fmt)  # set column width

    # set title 
    header_format = workbook.add_format({'bold': True,
                                     'text_wrap': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC', 
                                     'border': 1})      
    worksheet_cols = df2.shape[1]
    range_str = 'A1:'+chr(ord('A')+worksheet_cols)+'1'
    worksheet.merge_range(range_str,title,header_format)     

In [ ]:
writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
df2.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook

title = '应收账款汇总'+month
handle_worksheet_format(writer, df2, title)

writer.save() #saves workbook to file in python file directory
writer.close()

In [ ]:
print('success')